<a href="https://colab.research.google.com/github/Insidiae/ztm-machine-learning/blob/main/7_deep_learning_neural_networks_tensorflow_2/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐶 Using Transfer Learning and TensorFlow 2.0 to Classify Different Dog Breeds

This notebook builds an end-to-end multi-class image classifier using TensorFlow 2.x and TensorFlow Hub.

## 1. Problem

> Identify the breed of a dog given an image of a dog.

When I'm sitting at the cafe and I take a photo of a dog, I want to know what breed of dog it is.

## 2. Data

The data we're using is from [Kaggle's dog breed identification competition](https://www.kaggle.com/c/dog-breed-identification/data).

## 3. Evaluation
The evaluation is a file with prediction probabilities for each dog breed of each test image.

For more info, check out the [Kaggle dog breed identification competition page](https://www.kaggle.com/c/dog-breed-identification/overview/evaluation).

## 4. Features

Some information about the data:

- We're dealing with images (unstructured data) so it's probably best we use deep learning/transfer learning.
- There are 120 breeds of dogs (this means there are 120 different classes).
- There are around 10,000+ images in the training set (these images have labels).
- There are around 10,000+ images in the test set (these images have no labels, because we'll want to predict them).

### 4.1. Getting our workspace ready

For this project, we'll be using [Google Colaboratory](https://colab.research.google.com/) to host our workspace.

Here's a basic checklist of what we need to set up:

- Import TensorFlow 2.x ✅

In [1]:
import tensorflow as tf
print("TF version:", tf.__version__)

TF version: 2.6.0


- Import TensorFlow Hub ✅

In [2]:
import tensorflow_hub as hub
print("Hub version:", hub.__version__)

Hub version: 0.12.0


- Make sure we're using a GPU ✅

In [3]:
print("GPU:", "Available (YESSS!!!!!)" if tf.config.list_physical_devices("GPU") else "Not available...")

GPU: Available (YESSS!!!!!)


### 4.2. Getting data ready

We'll be using data from the [Kaggle dog breed identification competition](https://www.kaggle.com/c/dog-breed-identification/overview). It consists of a collection of 10,000+ labelled images of 120 different dog breeds.

This project is hosted via Google Colaboratory. As such, one great way to setup our project data is to mount our Google Drive folder and import the data from there:

In [4]:
# Running this cell will provide you with a token to link your drive to this notebook
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We'll first have to unzip the files provided by Kaggle:

> **Note:** Running the cell below for the first time could take a while (a couple of minutes is normal). After you've run it once and got the data in your Google Drive, you don't need to run it again.

> **Note 2:** Wherever you see something like `drive/MyDrive/folderPath/` you will need to change it to wherever you are storing your files for this project. The first place you'll have to change it is the cell below:

In [5]:
# Use the '-d' parameter as the destination for where the files should go
# !unzip "drive/MyDrive/ZTM-MLCourse/dog-breed-identification.zip" -d "drive/MyDrive/ZTM-MLCourse/"